In [1]:
import os
import warnings
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")
warnings.filterwarnings('ignore')

os.environ["SERPER_API_KEY"]=SERPER_API_KEY
os.environ['OPENAI_API_KEY']=OPENAI_API_KEY

In [2]:
from langchain_core.messages import AIMessage
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode

In [3]:
@tool
def get_weather(location: str):
    """Call to get the weather"""
    if location in ["서울", "인천"]:
        return "It's 60 degrees and foggy."
    else:
        return "It's 90 degrees and sunny."

@tool
def get_coolest_cities():
    """Get a list of coolest cities"""
    return "서울, 고성"

In [4]:
tools = [get_weather, get_coolest_cities]
tool_node = ToolNode(tools)

In [5]:
from langchain_openai import ChatOpenAI

model_with_tools = ChatOpenAI(
    model = "gpt-4o-mini", temperature=0
).bind_tools(tools)

In [6]:
model_with_tools.invoke("서울 날씨는 어때?").tool_calls

[{'name': 'get_weather',
  'args': {'location': '서울'},
  'id': 'call_WlLMz0FXz1JSQLxxrhTuBwmB',
  'type': 'tool_call'}]

In [7]:
model_with_tools.invoke("대한민국 대통령은 누구야?").tool_calls

[]

In [8]:
model_with_tools.invoke("한국에서 가장 추운 도시는?").tool_calls

[{'name': 'get_coolest_cities',
  'args': {},
  'id': 'call_ostMWTlTbtFTQS880yTwlMJo',
  'type': 'tool_call'}]

In [10]:
tool_node.invoke({"messages": [model_with_tools.invoke("서울 날씨는 어때?")]})

{'messages': [ToolMessage(content="It's 60 degrees and foggy.", name='get_weather', tool_call_id='call_MIpMXuONbSchq6HQgfEri8at')]}

In [13]:
from typing import Annotated, Literal, TypedDict
from langchain_core.messages import HumanMessage
from langchain_core.tools import tool
from langgraph.graph import END, START, StateGraph, MessagesState

def should_continue(state: MessagesState) -> Literal["tools", END]:
    messages = state['messages']
    last_message = messages[-1]
    if last_message.tool_calls:
        return "tools"
    return END

def call_model(state: MessagesState):
    messages = state["messages"]
    response = model_with_tools.invoke(messages)
    return {"messages": [response]}

workflow = StateGraph(MessagesState)

workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

workflow.add_edge(START, "agent")
workflow.add_conditional_edges(
    "agent",
    should_continue
)

workflow.add_edge("tools", "agent")

app = workflow.compile()

In [14]:
final_state = app.invoke(
    {"messages": [HumanMessage(content="서울의 날씨는 어때?")]},
)
final_state["messages"][-1].content

'서울의 날씨는 60도(약 15도)이며 안개가 낀 상태입니다.'

In [16]:
for chunk in app.stream(
    {"messages": [("human","가장 추운 도시의 날씨는 어때?")]},
    stream_mode="values"
):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

가장 추운 도시의 날씨는 어때?
================================== Ai Message ==================================
Tool Calls:
  get_coolest_cities (call_2Ta02ggmPRPYSNhigZP6zL1z)
 Call ID: call_2Ta02ggmPRPYSNhigZP6zL1z
  Args:
================================= Tool Message =================================
Name: get_coolest_cities

서울, 고성
================================== Ai Message ==================================
Tool Calls:
  get_weather (call_8fROebAcLimxlXSNns9SXuRK)
 Call ID: call_8fROebAcLimxlXSNns9SXuRK
  Args:
    location: 서울
  get_weather (call_rUzc9BMOUq9fDkX93Vtv1bRo)
 Call ID: call_rUzc9BMOUq9fDkX93Vtv1bRo
  Args:
    location: 고성
================================= Tool Message =================================
Name: get_weather

It's 90 degrees and sunny.
================================== Ai Message ==================================

가장 추운 도시인 서울의 날씨는 60도(약 15도)로 안개가 끼어 있습니다. 반면, 고성의 날씨는 90도(약 32도)로 맑

: 